<a href="https://colab.research.google.com/github/AIRJK/CPA-Attack/blob/master/2_Order_CPA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

数据预处理


In [ ]:
import os
from google.colab import drive
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats
import time
import csv
start = time.process_time()

file_old = 'drive/My Drive/2_CpaRawData/7M_Filter/Plaintext.csv'
file_temp = 'drive/My Drive/2_CpaRawData/7M_Filter/Plaintext2.csv'
 
with open(file_old, 'r', newline='', encoding='gbk') as f_old, \
    open(file_temp, 'w', newline='', encoding='gbk') as f_temp:
    f_csv_old = csv.reader(f_old)
    f_csv_temp = csv.writer(f_temp)
    i = 0
    for rows in f_csv_old:
      i += 1
      if i == 3:                  
        f_csv_temp.writerow(rows)
      if i == 4:
        i = 0
        
plain = np.loadtxt('drive/My Drive/2_CpaRawData/7M_Filter/Plaintext2.csv',dtype=str)
print(plain)

def readfile(num):
    tmp = np.loadtxt('drive/My Drive/2_CpaRawData/7M_Filter/Trace00%04d.csv' % num, delimiter="," ,usecols=(1,), dtype=float)
    tmp.T
    # print(tmp)
    return tmp



# print(plaintxt)

power = np.empty([1200,1000],dtype = float)
for i in range(1,1201):
  tmp = readfile(i)
  for j in range(1000):
    power[i-1][j] = float(tmp[j])
  print("处理第%d条波形中"%(i-1))

print("波形处理完毕")
# print(type(power[0][0]))

# print(power)
plaintxt = np.empty([1200,16],dtype = int)
for i in range(1200):
  for j in range(16):
    plaintxt[i][j] = int(plain[i][j],16)
  print("转换第%d条明文中" %(i))



np.savetxt('drive/My Drive/2_CpaRawData/7Mplain.txt', plaintxt, fmt = '%d')
np.savetxt('drive/My Drive/2_CpaRawData/7Mpower.txt', power, fmt = '%f')

    

2-CPA


CODE TEST2


In [8]:
import os
from google.colab import drive
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats
import time
import csv
from numba import jit



start = time.process_time()

power = np.loadtxt("drive/My Drive/2_CpaRawData/Wpower.txt", dtype=float)
plain = np.loadtxt("drive/My Drive/2_CpaRawData/Wplain.txt", dtype=np.int)




sbox = np.array([0x63, 0x7C, 0x77, 0x7B, 0xF2, 0x6B, 0x6F, 0xC5, 0x30, 0x01, 0x67, 0x2B, 0xFE, 0xD7, 0xAB, 0x76,
0xCA, 0x82, 0xC9, 0x7D, 0xFA, 0x59, 0x47, 0xF0, 0xAD, 0xD4, 0xA2, 0xAF, 0x9C, 0xA4, 0x72, 0xC0,
0xB7, 0xFD, 0x93, 0x26, 0x36, 0x3F, 0xF7, 0xCC, 0x34, 0xA5, 0xE5, 0xF1, 0x71, 0xD8, 0x31, 0x15,
0x04, 0xC7, 0x23, 0xC3, 0x18, 0x96, 0x05, 0x9A, 0x07, 0x12, 0x80, 0xE2, 0xEB, 0x27, 0xB2, 0x75,
0x09, 0x83, 0x2C, 0x1A, 0x1B, 0x6E, 0x5A, 0xA0, 0x52, 0x3B, 0xD6, 0xB3, 0x29, 0xE3, 0x2F, 0x84,
0x53, 0xD1, 0x00, 0xED, 0x20, 0xFC, 0xB1, 0x5B, 0x6A, 0xCB, 0xBE, 0x39, 0x4A, 0x4C, 0x58, 0xCF,
0xD0, 0xEF, 0xAA, 0xFB, 0x43, 0x4D, 0x33, 0x85, 0x45, 0xF9, 0x02, 0x7F, 0x50, 0x3C, 0x9F, 0xA8,
0x51, 0xA3, 0x40, 0x8F, 0x92, 0x9D, 0x38, 0xF5, 0xBC, 0xB6, 0xDA, 0x21, 0x10, 0xFF, 0xF3, 0xD2,
0xCD, 0x0C, 0x13, 0xEC, 0x5F, 0x97, 0x44, 0x17, 0xC4, 0xA7, 0x7E, 0x3D, 0x64, 0x5D, 0x19, 0x73,
0x60, 0x81, 0x4F, 0xDC, 0x22, 0x2A, 0x90, 0x88, 0x46, 0xEE, 0xB8, 0x14, 0xDE, 0x5E, 0x0B, 0xDB,
0xE0, 0x32, 0x3A, 0x0A, 0x49, 0x06, 0x24, 0x5C, 0xC2, 0xD3, 0xAC, 0x62, 0x91, 0x95, 0xE4, 0x79,
0xE7, 0xC8, 0x37, 0x6D, 0x8D, 0xD5, 0x4E, 0xA9, 0x6C, 0x56, 0xF4, 0xEA, 0x65, 0x7A, 0xAE, 0x08,
0xBA, 0x78, 0x25, 0x2E, 0x1C, 0xA6, 0xB4, 0xC6, 0xE8, 0xDD, 0x74, 0x1F, 0x4B, 0xBD, 0x8B, 0x8A,
0x70, 0x3E, 0xB5, 0x66, 0x48, 0x03, 0xF6, 0x0E, 0x61, 0x35, 0x57, 0xB9, 0x86, 0xC1, 0x1D, 0x9E,
0xE1, 0xF8, 0x98, 0x11, 0x69, 0xD9, 0x8E, 0x94, 0x9B, 0x1E, 0x87, 0xE9, 0xCE, 0x55, 0x28, 0xDF,
0x8C, 0xA1, 0x89, 0x0D, 0xBF, 0xE6, 0x42, 0x68, 0x41, 0x99, 0x2D, 0x0F, 0xB0, 0x54, 0xBB, 0x16
], dtype = np.uint8)



@jit
def hamming_weight(x):
    n = x
    c = 0
    while(n != 0):
      n = n&(n-1)
      c+=1
    return c
@jit
def MyCorr(x, y):
  sum1 = sum(x)
  sum2 = sum(y)
  avg1 = sum1/2000
  avg2 = sum2/2000
  sum1 = 0
  sum2 = 0
  sum3 = 0
  for i in range(2000):
    sum1 += (x[i]-avg1)*(y[i]-avg2)
    sum2 += (x[i]-avg1)**2
    sum3 += (y[i]-avg2)**2
  sum1 /= 2000
  sum2 /= 2000
  sum3 /= 2000
  res = sum1/np.sqrt(sum2 * sum3)
  return res


def CPA2(num1, num2, p1, p2, plain, power):
  HW = np.empty((256, 256, 2000), dtype=np.uint8)

  for g1 in range(256):
      for g2 in range(256):
          for l in range(2000):
              s00 = plain[l][num1] ^ g1
              s01 = sbox[s00]
              s10 = plain[l][num2] ^ g2
              s11 = sbox[s10]
              HW[g1][g2][l] = hamming_weight(s01 ^ s11)

          print("%d %d"%(g1, g2))
  pre_power = np.empty((2000),dtype = np.double)
  for l in range(2000):
    pre_power[l] = abs(power[l][p1] - power[l][p2])

      
  maxcorr = 0
  x = []
  key1 = 0
  key2 = 0
  for i in range(256):
    for j in range(256):
      for k in range(2000):
        x.append(HW[i][j][k])
      corr = MyCorr(HW[i][j],pre_power)
      if(corr > maxcorr):
        maxcorr = corr
        key1 = i
        key2 = j

      print("key1: "+str(i)+" key2: "+str(j))
      x.clear()

  print("密钥的第%d字节为: 0x%02x 密钥的第%d字节为: 0x%02x"%(num1, key1, num2, key2))




CPA2(0,1,378,878, plain, power)
end = time.process_time()

print("执行时长："+str(end-start))

流式输出内容被截断，只能显示最后 5000 行内容。
236 121
236 122
236 123
236 124
236 125
236 126
236 127
236 128
236 129
236 130
236 131
236 132
236 133
236 134
236 135
236 136
236 137
236 138
236 139
236 140
236 141
236 142
236 143
236 144
236 145
236 146
236 147
236 148
236 149
236 150
236 151
236 152
236 153
236 154
236 155
236 156
236 157
236 158
236 159
236 160
236 161
236 162
236 163
236 164
236 165
236 166
236 167
236 168
236 169
236 170
236 171
236 172
236 173
236 174
236 175
236 176
236 177
236 178
236 179
236 180
236 181
236 182
236 183
236 184
236 185
236 186
236 187
236 188
236 189
236 190
236 191
236 192
236 193
236 194
236 195
236 196
236 197
236 198
236 199
236 200
236 201
236 202
236 203
236 204
236 205
236 206
236 207
236 208
236 209
236 210
236 211
236 212
236 213
236 214
236 215
236 216
236 217
236 218
236 219
236 220
236 221
236 222
236 223
236 224
236 225
236 226
236 227
236 228
236 229
236 230
236 231
236 232
236 233
236 234
236 235
236 236
236 237
236 238
236 239
236 240
236 241
236 2

<ipython-input-8-a0f4a4824745>:48: NumbaWarning: 
Compilation is falling back to object mode WITH looplifting enabled because Function "MyCorr" failed type inference due to: Untyped global name 'sum': cannot determine Numba type of <class 'builtin_function_or_method'>

File "<ipython-input-8-a0f4a4824745>", line 50:
def MyCorr(x, y):
  sum1 = sum(x)
  ^

  @jit
<ipython-input-8-a0f4a4824745>:48: NumbaWarning: 
Compilation is falling back to object mode WITHOUT looplifting enabled because Function "MyCorr" failed type inference due to: Untyped global name 'sum': cannot determine Numba type of <class 'builtin_function_or_method'>

File "<ipython-input-8-a0f4a4824745>", line 50:
def MyCorr(x, y):
  sum1 = sum(x)
  ^

  @jit
/usr/local/lib/python3.6/dist-packages/numba/object_mode_passes.py:178: NumbaWarning: Function "MyCorr" was compiled in object mode without forceobj=True, but has lifted loops.

File "<ipython-input-8-a0f4a4824745>", line 50:
def MyCorr(x, y):
  sum1 = sum(x)
  ^

  st

流式输出内容被截断，只能显示最后 5000 行内容。
key1: 236 key2: 123
key1: 236 key2: 124
key1: 236 key2: 125
key1: 236 key2: 126
key1: 236 key2: 127
key1: 236 key2: 128
key1: 236 key2: 129
key1: 236 key2: 130
key1: 236 key2: 131
key1: 236 key2: 132
key1: 236 key2: 133
key1: 236 key2: 134
key1: 236 key2: 135
key1: 236 key2: 136
key1: 236 key2: 137
key1: 236 key2: 138
key1: 236 key2: 139
key1: 236 key2: 140
key1: 236 key2: 141
key1: 236 key2: 142
key1: 236 key2: 143
key1: 236 key2: 144
key1: 236 key2: 145
key1: 236 key2: 146
key1: 236 key2: 147
key1: 236 key2: 148
key1: 236 key2: 149
key1: 236 key2: 150
key1: 236 key2: 151
key1: 236 key2: 152
key1: 236 key2: 153
key1: 236 key2: 154
key1: 236 key2: 155
key1: 236 key2: 156
key1: 236 key2: 157
key1: 236 key2: 158
key1: 236 key2: 159
key1: 236 key2: 160
key1: 236 key2: 161
key1: 236 key2: 162
key1: 236 key2: 163
key1: 236 key2: 164
key1: 236 key2: 165
key1: 236 key2: 166
key1: 236 key2: 167
key1: 236 key2: 168
key1: 236 key2: 169
key1: 236 key2: 170
key1: 236 key